## 3.1 추론 기반 기법과 신경망
단어를 벡터로 표현하는 방법은 크게 '통계 기반 기법'과 '추론 기반 기법'이 있다. 의미를 얻는 방식은 서로 크게 다르지만, 그 배경에는 분포가설이 있다.

### 3.1.1 통계 기반 기법의 문제점
통계기반기법은 단어의 동시발생 행렬을 만들고 그 행렬에 SVD(특잇값 분해)를 적용해 밀집벡터를 얻었다.  
그렇지만 이 방식은 대규모 말뭉치를 다룰 때 문제가 발생한다. 어휘가 100만개라면 100만*100만의 행렬에 [SVD](https://angeloyeo.github.io/2019/08/01/SVD.html)를 적용해야 하는데, n\*n의 계산비용은 $O(n^3)$이다.

### 3.1.2. 추론 기반 기법 개요
추론 기반 기법은 당연히 '추론'이 주된 작업이다. 아래 그림처럼 주변 단어가 주어졌을 때 '?'에 무슨 단어가 들어가는지를 추측하는 작업이다.
![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%203-2.png?raw=true)  
이러한 추론 문제를 반복해서 풀면서 단어의 출현 패턴을 학습하는 것이다.

추론기반기법에서는 어떠한 모델이 등장하는데, 이 모델로 신경망을 사용할 것이다.

### 3.1.3 신경망에서의 단어처리
신경망을 이용해 '단어'를 처리한다. 단어를 그대로 처리할 수 없으니 고정길이의 벡터인 원핫표현으로 변환한다.
따라서 총 어휘 수 만큼의 원소를 갖는 벡터를 준비하고, 인덱스가 단어 ID와 같은 원소를 1로, 나머지는 0으로 설정한다.
![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%203-5.png?raw=true)

![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%203-6.png?raw=true)  
완전 계층에 의한 변환: 입력층의 각 뉴런은 7개의 단어 각각에 대응(은닉층 뉴런은 3개)

완전연결계층에 의한 변환은 파이썬으로 다음과 같이 작성할 수 있다.

In [1]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])   # 입력
W = np.random.randn(7, 3)               # 가중치
h = np.matmul(c, W)                     # 중간 노드
print(h)

[[-0.00451681 -0.14811647 -0.1415103 ]]


In [2]:
W

array([[-0.00451681, -0.14811647, -0.1415103 ],
       [-0.61190463,  0.81887391,  0.18460785],
       [ 1.67094574,  0.93669359, -1.02195061],
       [-0.91966896, -0.90614523,  1.24577705],
       [-1.90944386,  0.48653443, -0.35423597],
       [-3.80064972,  0.71340245, -1.13108254],
       [-0.71483301,  0.44261068,  0.18541838]])

![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%203-8.png?raw=true0)

In [3]:
# MatMul
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None

    def forward(self, x):
        W, = self.params
        out = np.matmul(x, W)
        self.x = x
        return out

    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        self.grads[0][...] = dW     # grads[0]처럼 쓰면 얕은 복사, grads[0][...]처럼 쓰면 깊은 복사가 이루어진다.
        return dx

In [4]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])   # 입력
W = np.random.randn(7, 3)               # 가중치
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-0.35521846  1.14171451 -0.45773268]]
